In [1]:
import pytorch_lightning

from Reimplementation.data.DeepCDRDataModule import DeepCDRDataModule
from Reimplementation.models.DeepCDRLitModule import DeepCDRLitModule
from Reimplementation.util.config import *
%load_ext tensorboard

In [2]:
dpath = '../data'
drug_info_file = '%s/GDSC/1.Drug_listMon Jun 24 09_00_55 2019.csv' % dpath
cell_line_info_file = '%s/CCLE/Cell_lines_annotations_20181226.txt' % dpath
drug_feature_file = '%s/GDSC/drug_graph_feat' % dpath
genomic_mutation_file = '%s/CCLE/genomic_mutation_34673_demap_features.csv' % dpath
cancer_response_exp_file = '%s/CCLE/GDSC_IC50.csv' % dpath
gene_expression_file = '%s/CCLE/genomic_expression_561celllines_697genes_demap_features.csv' % dpath
methylation_file = '%s/CCLE/genomic_methylation_561celllines_808genes_demap_features.csv' % dpath
raw_files = NeededFiles(drug_info_file, cell_line_info_file, drug_feature_file, genomic_mutation_file,
                        cancer_response_exp_file, gene_expression_file, methylation_file)

GCN = ConvType.GCN
GAT = ConvType.GAT
tanh = ActivationFunc.Tanh
relu = ActivationFunc.ReLU
f_tanh = FuncActivationFunc.Tanh
f_relu = FuncActivationFunc.ReLU

drug_config = GNNConfig([75, 256, 128, 64, 20], [GCN, GCN, GCN, GCN], [f_relu, f_relu, f_relu, f_relu], False)
mutation_config = Conv1DConfig([1, 30, 30, 30], [700, 500, 100], [tanh, relu, relu], [20, 10, 10], False)
g_expr_config = FCConfig([697, 256, 20], [tanh, relu], False)
meth_config = FCConfig([808, 256, 20], [tanh, relu], False)
regressor_config = Conv1DConfig([1, 30, 10, 5], [150, 5, 5], [tanh, tanh, tanh], [2, 3, 3], False)
config = DeepCDRConfig(drug_config, mutation_config, g_expr_config, meth_config, regressor_config)

In [3]:
data = DeepCDRDataModule(raw_files, 100, batch_size=30)
model = DeepCDRLitModule(config)

In [4]:
batch = data.get_debug_batch()
drug, mut, g_exp, meth, ic50 = batch
print(mut.shape)

torch.Size([30, 1, 34673])


In [12]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

pl.seed_everything(0)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(monitor="Validation loss", mode='min', stopping_threshold=0.005, patience=4)
logger=TensorBoardLogger(save_dir="log", name="DeepCDR_Logs", version="Only one final Linear Layer for regression with convolution")
trainer = pl.Trainer(logger=logger, callbacks=[early_stopping], log_every_n_steps=1, auto_lr_find=True, auto_scale_batch_size=True, min_epochs=40)
debug_trainer = pl.Trainer(overfit_batches=1, logger=logger, max_epochs=100, log_every_n_steps=1)

Global seed set to 0
/home/user/PycharmProjects/DeepCDR/venv/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.


In [13]:
# writer = SummaryWriter()
# writer.add_graph(model.regressor, batch[2])
# writer.close()
debug_trainer.fit(model, datamodule=data)
# trainer.tune(model, datamodule=data)
# trainer.fit(model, datamodule=data)


  | Name              | Type              | Params | In sizes              | Out sizes  
----------------------------------------------------------------------------------------------
0 | drug_gnn          | Sequential_841b95 | 61.9 K | [[100, 75], [2, 101]] | [100, 20]  
1 | mutation_cnn1d    | Sequential        | 561 K  | [1, 1, 34673]         | [1, 30, 2] 
2 | g_expr_fcn        | Sequential        | 183 K  | [1, 697]              | [1, 20]    
3 | meth_fcn          | Sequential        | 212 K  | [1, 808]              | [1, 20]    
4 | regression_conv1d | Sequential        | 6.3 K  | [1, 1, 2140]          | [1, 5, 108]
----------------------------------------------------------------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.101     Total estimated model params size (MB)
/home/user/PycharmProjects/DeepCDR/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:471: UserWarning: You 

Training: 0it [00:00, ?it/s]

In [11]:
# Loading TensorBoard
%reload_ext tensorboard
%tensorboard --logdir log/

Launching TensorBoard...